In [1]:
%pip install --upgrade keras

In [2]:
 %pip install torch torchvision torchaudio

In [3]:
%pip install scikit-learn pandas matplotlib transformers -U

In [4]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
print(keras.__version__)

3.2.1


In [5]:
# стандартные библиотеки
import os, re
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import Counter
from string import punctuation
import matplotlib.pyplot as plt
%matplotlib inline


import keras, torch
from transformers import AutoModel
from transformers import AutoTokenizer

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lenta_sample.csv')
data.dropna(subset=['topic', 'text'], inplace=True)

In [ ]:
# №3 Дообучение без замораживания Twitter/twhin-bert-base

In [8]:
tokenizer = AutoTokenizer.from_pretrained("Twitter/twhin-bert-base")
model_bert = AutoModel.from_pretrained("Twitter/twhin-bert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# переводим индекс токена обратно в текст
encoded = tokenizer.encode(data.loc[4, 'title'])
[tokenizer.decode([x]) for x in encoded]

['<s>',
 'По',
 'факт',
 'у',
 'пожар',
 'а',
 'на',
 'завод',
 'е',
 'в',
 'Ю',
 'гр',
 'е',
 'за',
 'ведено',
 'дело',
 '</s>']

In [10]:
text = data.loc[4, 'title']
text_ids = tokenizer.encode(text, add_special_tokens=True,
                            return_tensors='pt') # преобразуем в торч формат
output = model_bert(text_ids, output_attentions=False)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [11]:
X = []

for text in data.title:
    ids = tokenizer.encode(text)

    X.append(ids[:512])

X = keras.preprocessing.sequence.pad_sequences(X, padding='post', maxlen=512)
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = keras.utils.to_categorical([label2id[label] for label in data.topic.values])

train_index, valid_index = train_test_split(list(range(len(X))), test_size=0.05, stratify=data.topic)
X_train, y_train = X[train_index], y[train_index]
X_valid, y_valid = X[valid_index], y[valid_index]

In [12]:
model_bert = AutoModel.from_pretrained('Twitter/twhin-bert-base')

Some weights of BertModel were not initialized from the model checkpoint at Twitter/twhin-bert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
class bert_wrapper(keras.Model):
    def __init__(self, model_bert, **kwargs):
        super().__init__(**kwargs)
        # модель нужно обернуть в специальный класс чтобы она была совместима с другими керас слоями
        self.model_bert = keras.layers.TorchModuleWrapper(model_bert)

    def call(self, inputs):
        x = self.model_bert(inputs)
        return x[0]

In [15]:
input_word_ids = keras.layers.Input(shape=(512,), dtype='int32',
                                     name="input_ids")

output = bert_wrapper(model_bert)(input_word_ids)
# берем только самый первый вектор (CLS токен)
drop = keras.layers.Dropout(0.1)(output[:, 0])
dense = keras.layers.Dense(y.shape[1], activation='softmax')(drop)

model_clf = keras.Model(inputs=input_word_ids, outputs=dense)


model_clf.compile(keras.optimizers.Adam(learning_rate=2e-5,
                                    ),
                  loss='categorical_crossentropy',
                  metrics=['accuracy',
                           keras.metrics.RecallAtPrecision(0.80, name='rec_prec')])

In [16]:
if torch.cuda.is_available():
    model_clf.to('cuda')

In [ ]:
model_clf.fit(torch.LongTensor(X_train), torch.LongTensor(y_train),
          validation_data=(torch.LongTensor(X_valid), torch.LongTensor(y_valid)),
          batch_size=4,
          epochs=5) # уменьшила кол-во эпох

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:588: UserWarning: Gradients do not exist for variables ['variable_1', 'variable_209', 'variable_210'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


144/144 ━━━━━━━━━━━━━━━━━━━━ 4954s 34s/step - accuracy: 0.0675 - loss: 3.0855 - rec_prec: 0.0000e+00 - val_accuracy: 0.0645 - val_loss: 2.7623 - val_rec_prec: 0.0000e+00
Epoch 2/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 4913s 34s/step - accuracy: 0.0564 - loss: 2.8040 - rec_prec: 0.0000e+00 - val_accuracy: 0.0968 - val_loss: 2.7273 - val_rec_prec: 0.0000e+00
Epoch 3/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 4898s 34s/step - accuracy: 0.0789 - loss: 2.7851 - rec_prec: 0.0011 - val_accuracy: 0.0968 - val_loss: 2.7185 - val_rec_prec: 0.0000e+00
Epoch 4/5
144/144 ━━━━━━━━━━━━━━━━━━━━ 4898s 34s/step - accuracy: 0.0837 - loss: 2.7410 - rec_prec: 0.0000e+00 - val_accuracy: 0.0968 - val_loss: 2.7183 - val_rec_prec: 0.0000e+00
Epoch 5/5
 97/144 ━━━━━━━━━━━━━━━━━━━━ 26:31 34s/step - accuracy: 0.0896 - loss: 2.7692 - rec_prec: 0.0028

In [ ]:
# я сдаюсь, я уже в третий раз считаю это и у меня слетает коллаб, как бы я его не мониторила

In [1]:
print(model_clf.history.history.keys())
plt.plot(model_clf.history.history['accuracy'])
plt.plot(model_clf.history.history['val_accuracy'])
plt.title('model rec_prec')
plt.ylabel('rec_prec')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'model_clf' is not defined

In [ ]:
pred = model_clf.predict(X_valid, batch_size=10).argmax(1)
print(classification_report(y_valid.argmax(1), pred, labels=list(range(len(label2id))),
                            target_names=list(label2id), zero_division=0))